In [2]:
!pip install ical

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.9/434.9 kB 2.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [179]:
import pandas as pd
import json
import pandas as pd
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
tqdm.pandas()
import time
import re
from datetime import date
from datetime import datetime
from ical.calendar import Calendar
from ical.event import Event
import pytz
from datetime import datetime, timedelta
from ical.calendar_stream import IcsCalendarStream

### Getting JSON

In [18]:
def get_schedule(group: str) -> any:
    '''

    '''
    url = f'https://rasp.dmami.ru/site/group?group={group}&session=0'
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:130.0) Gecko/20100101 Firefox/130.0',
        'Accept': '*/*',
        'Accept-Language': 'ru,en-US;q=0.7,en;q=0.3',
        'X-Requested-With': 'XMLHttpRequest',
        'Sec-Fetch-Dest': 'empty',
        'Sec-Fetch-Mode': 'cors',
        'Sec-Fetch-Site': 'same-origin',
        'Priority': 'u=0',
        'Referer': 'https://rasp.dmami.ru/'
    }

    response = requests.get(url, headers=headers, cookies={'session': '0'})

    return response.json()


In [19]:
file = get_schedule('242-621')
file

{'status': 'ok',
 'grid': {'1': {'1': [],
   '2': [],
   '3': [],
   '4': [],
   '5': [{'sbj': 'Линейная алгебра',
     'teacher': 'Богданов Михаил Рифкатович',
     'dts': '07 Окт - 20 Окт',
     'df': '2024-10-07',
     'dt': '2024-10-20',
     'auditories': [{'title': '<a href="https://online.mospolytech.ru/" target="_blank">🏠 СДО (LMS)</a>',
       'color': '#00F'}],
     'shortRooms': [],
     'location': 'СДО (LMS)',
     'type': 'Лекция эор',
     'week': '',
     'align': 'center',
     'e_link': None}],
   '6': [{'sbj': 'Экономическая теория',
     'teacher': 'Шкаленко Анна Викторовна ',
     'dts': '07 Окт - 20 Окт',
     'df': '2024-10-07',
     'dt': '2024-10-20',
     'auditories': [{'title': '<a href="https://online.mospolytech.ru/" target="_blank">🏠 СДО (LMS)</a>',
       'color': '#00F'}],
     'shortRooms': [],
     'location': 'СДО (LMS)',
     'type': 'Лекция',
     'week': '',
     'align': 'center',
     'e_link': None},
    {'sbj': 'Линейная алгебра',
     'teache

### Converting JSON to pd.DataFrame()

In [123]:
def json_to_df(file):
        
    def dat(date_range_str):
        # Словарь для преобразования названий месяцев на русском языке
        months = {
            "Янв": 1, "Фев": 2, "Мар": 3, "Апр": 4,
            "Май": 5, "Июн": 6, "Июл": 7, "Авг": 8,
            "Сен": 9, "Окт": 10, "Ноя": 11, "Дек": 12
        }
        
        # Разделяем строку на две даты
        start_str, end_str = date_range_str.split(" - ")
        
        # Преобразуем строки с датами в объекты datetime
        def parse_date(date_str):
            day, month_abbr = date_str.split()
            month = months[month_abbr]
            return datetime(year=datetime.now().year, month=month, day=int(day))
           
        start_date = parse_date(start_str)
        end_date = parse_date(end_str)
        date_lesson = [start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d')]
        return date_lesson
        
    lessons = {
                1: '9:00-10:30',
                2: '10:40-12:10',
                3: '12:20-13:50',
                4: '14:30-16:00',
                5: '16:10-17:40',
                6: '18:20-19:40',
                7: '19:50-21:10',
            }
    
    all = pd.DataFrame()
    combined_df = pd.DataFrame()
    days = [' ', 'ПН', 'ВТ', 'СР', 'ЧТ', 'ПТ', 'СБ', 'ВС']
    data = pd.DataFrame(file['grid'])

    for i in range(1, 7):
        for j in range(1, 8):
            # Создаем DataFrame из JSON-данных
            temp_df = pd.DataFrame(file['grid'][str(i)][str(j)])
            # Добавляем столбцы 'lesson' и 'day'
            temp_df['lesson'] = j
            temp_df['day'] = days[i]
            # Объединяем с основным DataFrame
            combined_df = pd.concat([combined_df, temp_df], ignore_index=True)

    combined_df = combined_df.iloc[:, [0,1,2,3, 4, 9, 10]]

    combined_df['dts'] = combined_df['dts'].apply(dat)
    def extract_times(lesson_number):
        time_range = lessons[lesson_number]  # Получаем строку с временем
        start_time, end_time = time_range.split('-')  # Разделяем время на начало и конец
        return [start_time, end_time]  # Возвращаем список
    
    # Создаем новый столбец со списком времени
    combined_df['time_range'] = combined_df['lesson'].apply(extract_times)

    return combined_df

In [124]:
df = json_to_df(file)
df

,lesson,day,sbj,teacher,dts,location,type,time_range
0,5,ПН,Линейная алгебра,Богданов Михаил Рифкатович,"[2024-10-07, 2024-10-20]",СДО (LMS),Лекция эор,"[16:10, 17:40]"
1,6,ПН,Экономическая теория,Шкаленко Анна Викторовна,"[2024-10-07, 2024-10-20]",СДО (LMS),Лекция,"[18:20, 19:40]"
2,6,ПН,Линейная алгебра,Богданов Михаил Рифкатович,"[2024-10-21, 2024-11-03]",СДО (LMS),Практика эор,"[18:20, 19:40]"
3,6,ПН,История России,Азаматова Гульназ Булатовна,"[2024-11-11, 2024-12-08]",Webinar,Лекция,"[18:20, 19:40]"
4,6,ПН,Цифровая грамотность https://online.mospolytec...,Меньшикова Наталия Павловна,"[2024-12-09, 2024-12-14]",Online курс,Лекция,"[18:20, 19:40]"
5,6,ПН,Цифровая грамотность,Меньшикова Наталия Павловна,"[2024-12-16, 2024-12-22]",Online курс,Лекция эор,"[18:20, 19:40]"
6,6,ПН,Экономическая теория,Шкаленко Анна Викторовна,"[2024-12-23, 2024-01-04]",СДО (LMS),Практика,"[18:20, 19:40]"
7,7,ПН,Экономическая теория,Шкаленко Анна Викторовна,"[2024-10-07, 2024-10-20]",СДО (LMS),Лекция,"[19:50, 21:10]"
8,7,ПН,Линейная алгебра,Богданов Михаил Рифкатович,"[2024-10-21, 2024-11-03]",СДО (LMS),Практика эор,"[19:50, 21:10]"
9,7,ПН,История России,Азаматова Гульназ Булатовна,"[2024-11-11, 2024-12-08]",СДО (LMS),Лекция эор,"[19:50, 21:10]"


### Generating ICal Calenedar from DataFrame

In [ ]:
def ical_gen(df):
    calendar = Calendar()
    days_mapping = {
    'ПН': 0, 'ВТ': 1, 'СР': 2, 'ЧТ': 3, 'ПТ': 4, 'СБ': 5, 'ВС': 6
    }
    timezone = pytz.timezone("Europe/Moscow")
    for index, rows in df.iterrows():
        calendar.events.append(
            Event(summary=rows[2], start=date(*list(map(lambda x: int(x), rows[4][0].split('-')))), end=date(*list(map(lambda x: int(x), rows[4][1].split('-')))), 
        )
        )

In [188]:
def ical_gen(df):
    calendar = Calendar()
    days_mapping = {
    'ПН': 0, 'ВТ': 1, 'СР': 2, 'ЧТ': 3, 'ПТ': 4, 'СБ': 5, 'ВС': 6
    }
    timezone = pytz.timezone("Europe/Moscow")
    for index, row in df.iterrows():
        start_date = datetime.strptime(row['dts'][0], '%Y-%m-%d')
        end_date = datetime.strptime(row['dts'][1], '%Y-%m-%d')
        day_of_week = days_mapping[row['day']]
    
        current_date = start_date
        while current_date <= end_date:
            if current_date.weekday() == day_of_week:
                start_time = datetime.strptime(row['time_range'][0], '%H:%M').time()
                end_time = datetime.strptime(row['time_range'][1], '%H:%M').time()

                start_datetime = datetime.combine(current_date, start_time)
                end_datetime = datetime.combine(current_date, end_time)

                start_datetime = timezone.localize(start_datetime)
                end_datetime = timezone.localize(end_datetime)
                calendar.events.append(
                    Event(summary = ' • '.join([row[2], row[5], row[3]]), dtstart = start_datetime, dtend = end_datetime, location = row['location'])
                )
            current_date += timedelta(days=1)


    with open(f'MPY_calendar.ics', 'w') as ics_file:
        ics_file.write(IcsCalendarStream.calendar_to_ics(calendar))
    print(f'Done! Calendar have saved')

In [189]:
ical_gen(df)

Done! Calendar have saved


In [193]:
for i, row in df.iterrows():
    
    start_time = datetime.strptime(row['time_range'][0], '%H:%M').time()
    end_time = datetime.strptime(row['time_range'][1], '%H:%M').time()
    print(row['time_range'][0], start_time, row['time_range'][1], end_time)

16:10 16:10:00 17:40 17:40:00
18:20 18:20:00 19:40 19:40:00
18:20 18:20:00 19:40 19:40:00
18:20 18:20:00 19:40 19:40:00
18:20 18:20:00 19:40 19:40:00
18:20 18:20:00 19:40 19:40:00
18:20 18:20:00 19:40 19:40:00
19:50 19:50:00 21:10 21:10:00
19:50 19:50:00 21:10 21:10:00
19:50 19:50:00 21:10 21:10:00
19:50 19:50:00 21:10 21:10:00
19:50 19:50:00 21:10 21:10:00
19:50 19:50:00 21:10 21:10:00
18:20 18:20:00 19:40 19:40:00
18:20 18:20:00 19:40 19:40:00
18:20 18:20:00 19:40 19:40:00
18:20 18:20:00 19:40 19:40:00
19:50 19:50:00 21:10 21:10:00
19:50 19:50:00 21:10 21:10:00
19:50 19:50:00 21:10 21:10:00
19:50 19:50:00 21:10 21:10:00
18:20 18:20:00 19:40 19:40:00
18:20 18:20:00 19:40 19:40:00
18:20 18:20:00 19:40 19:40:00
19:50 19:50:00 21:10 21:10:00
19:50 19:50:00 21:10 21:10:00
19:50 19:50:00 21:10 21:10:00
18:20 18:20:00 19:40 19:40:00
18:20 18:20:00 19:40 19:40:00
18:20 18:20:00 19:40 19:40:00
19:50 19:50:00 21:10 21:10:00
19:50 19:50:00 21:10 21:10:00
19:50 19:50:00 21:10 21:10:00
18:20 18:2